CCS なら厳しめのパラメタで daligner をやるだけでいけるんじゃないかということでやってみる。

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from IPython.display import display
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.io as pio
pio.templates.default = 'plotly_white'
import logging
import logzero
logzero.loglevel(logging.INFO)

In [2]:
from BITS.util.io import load_pickle, save_pickle
from BITS.util.proc import run_command
from BITS.seq.io import save_fasta
from BITS.plot.plotly import make_line, make_hist, make_scatter, make_layout, show_plot
import numpy as np
import pandas as pd
from logzero import logger
import numpy as np
import pandas as pd
import igraph as ig
from collections import Counter
import plotly.offline as py
import plotly.graph_objs as go

In [3]:
dir_fname = 'work_daligner_low_cover_rate/'
import os
os.chdir(dir_fname)

In [6]:
!(ls -l)

合計 4
-rw-r--r-- 1 yoshihiko_s users 194  9月 19 04:58 2019 DMEL_CCS_CEN.db
lrwxrwxrwx 1 yoshihiko_s users  50  9月 19 04:57 2019 centromere_reads_incl_low_cover_rate.fasta -> ../work/centromere_reads_incl_low_cover_rate.fasta


In [20]:
sync_reads = load_pickle("../work/centromere_reads_incl_low_cover_rate_all_sync.pkl")

## Run daligner

In [146]:
!(rm *.las)

In [7]:
!(HPC.daligner -T16 -k25 -w5 -l3000 -h60 -e.99 -s500 DMEL_CCS_CEN | bash -v)

# Daligner jobs (1)
daligner -k25 -w5 -h60 -e0.99 -l3000 -s500 -T16 DMEL_CCS_CEN DMEL_CCS_CEN && mv DMEL_CCS_CEN.DMEL_CCS_CEN.las DMEL_CCS_CEN.las
# Check initial .las files jobs (1) (optional but recommended)
LAcheck -vS DMEL_CCS_CEN DMEL_CCS_CEN
  DMEL_CCS_CEN: 243,368 all OK


In [8]:
!(LAshow -oc DMEL_CCS_CEN.db DMEL_CCS_CEN.las | head -10)


DMEL_CCS_CEN: 243,368 records

     1     107 c   [ 7,848..13,814> x <12,959.. 6,984]  (12,959 x 13,814 bps,13 trace pts)

         7848 
    A  =======+---------->           dif/(len1+len2) = 37/(5966+5975) =  0.62%
    B         <----------+=======
                           6984



In [9]:
!(LAdump -ocdl DMEL_CCS_CEN.db DMEL_CCS_CEN.las | head -10)

+ P 22972
% P 521
P 1 107 c .
L 13814 12959
C 7848 13814 0 5975
D 37
P 1 809 n .
L 13814 11653
C 0 9406 2240 11653
D 15


revcomp の時の b_read の start/end がややこしいので気をつける (LAshow と LAdump で数字が逆)

## Load overlaps from las

In [4]:
def las_to_ovlps(db_prefix):
    ovlps = []
    for line in run_command(f"LAdump -ocdl {db_prefix}.db {db_prefix}.las").split('\n'):
        if line.startswith('P'):
            _, a_read, b_read, strand, _ = line.strip().split(' ')
            a_read, b_read = map(int, (a_read, b_read))
        elif line.startswith('L'):
            _, a_len, b_len = line.strip().split(' ')
            a_len, b_len = map(int, (a_len, b_len))
        elif line.startswith('C'):
            _, a_start, a_end, b_start, b_end = line.strip().split(' ')
            a_start, a_end, b_start, b_end = map(int, (a_start, a_end, b_start, b_end))
        elif line.startswith('D'):
            _, n_diff = line.strip().split(' ')
            n_diff = int(n_diff)
            p_diff = round(n_diff / ((a_end - a_start + b_end - b_start) / 2) * 100, 2)   # in percentage
            ovlps.append((a_read, b_read, strand, a_start, a_end, a_len, b_start, b_end, b_len, n_diff, p_diff))
    return ovlps

## Construct a string graph from the filtered overlaps

In [5]:
def construct_string_graph(overlaps):
    nodes, edges = set(), set()
    for overlap in overlaps:
        f_id, g_id, strand, a_start, a_end, a_len, b_start, b_end, b_len, n_diff, p_diff = overlap
        
        if strand == 'c':
            # a_read[a_start:a_end] ~~ strand(b_read[b_start:b_end]) となるようにする
            # b の座標は常に b_read を forward に見た時のもの。配列の切り出しが先。revcomp は後。
            b_start, b_end = b_len - b_end, b_len - b_start
            
        if a_start > 0:
            if strand == 'n':
                if b_end == b_len:
                    """
                         f.B               f.E
                      f  ----------------->
                      g     --------->
                          g.B        g.E
                    """
                    overlap_type = "contains"
                else:
                    """
                         f.B         f.E
                      f  ----------->
                      g         ------------->
                                g.B           g.E
                    """
                    overlap_type = "suffix-prefix"
            else:
                if b_start == 0:
                    """
                         f.B               f.E
                      f  ----------------->
                      g     <---------
                          g.E        g.B
                    """
                    overlap_type = "contains"
                else:
                    """
                         f.B         f.E
                      f  ----------->
                      g         <-------------
                                g.E           g.B
                    """
                    overlap_type = "suffix-suffix"
        else:
            if a_end == a_len:
                overlap_type = "contained"
            else:
                if strand == 'n':
                    """
                                f.B         f.E
                      f          ----------->
                      g    ----------->
                         g.B          g.E
                    """
                    if b_start == 0:
                        overlap_type = "contains"
                    else:
                        overlap_type = "prefix-suffix"
                else:
                    """
                                f.B         f.E
                      f          ----------->
                      g    <-----------
                         g.E           g.B
                    """
                    if b_end == b_len:
                        overlap_type = "contains"
                    else:
                        overlap_type = "prefix-prefix"

        if overlap_type in ["contains", "contained"]:
            continue
        elif overlap_type == "suffix-prefix":
            nodes.update(["%s:B" % g_id,
                          "%s:B" % f_id,
                          "%s:E" % f_id,
                          "%s:E" % g_id])
            edges.update([("%s:B" % g_id, "%s:B" % f_id, a_start, p_diff),
                          ("%s:E" % f_id, "%s:E" % g_id, b_len - b_end, p_diff)])
        elif overlap_type == "suffix-suffix":
            nodes.update(["%s:E" % g_id,
                          "%s:B" % f_id,
                          "%s:E" % f_id,
                          "%s:B" % g_id])
            edges.update([("%s:E" % g_id, "%s:B" % f_id, a_start, p_diff),
                          ("%s:E" % f_id, "%s:B" % g_id, b_start, p_diff)])
        elif overlap_type == "prefix-suffix":
            nodes.update(["%s:B" % f_id,
                          "%s:B" % g_id,
                          "%s:E" % g_id,
                          "%s:E" % f_id])
            edges.update([("%s:B" % f_id, "%s:B" % g_id, b_start, p_diff),
                          ("%s:E" % g_id, "%s:E" % f_id, a_len - a_end, p_diff)])
        else:   # prefix-prefix
            nodes.update(["%s:B" % f_id,
                          "%s:E" % g_id,
                          "%s:B" % g_id,
                          "%s:E" % f_id])
            edges.update([("%s:B" % f_id, "%s:E" % g_id, b_len - b_end, p_diff),
                          ("%s:B" % g_id, "%s:E" % f_id, a_len - a_end, p_diff)])

    return ig.Graph.DictList(edges=(dict(source=s, target=t, length=l, diff=p) for s, t, l, p in edges),
                             vertices=None,
                             directed=True)

def transitive_reduction(sg):
    v_mark = ["vacant" for v in sg.vs]
    e_reduce = {e.tuple: False for e in sg.es}
    FUZZ = 10   # in bp; this length is in general shorter than unit length, thus we accept no unit shifts

    for v in sg.vs:
        if v.outdegree() == 0:
            continue

        oes = sorted(sg.es.select(_source=v.index), key=lambda x: x["length"])
        longest = oes[-1]["length"] + FUZZ
        for oe in oes:
            v_mark[oe.target] = "inplay"

        for oe in oes:
            if v_mark[oe.target] == "inplay":
                ooes = sorted(sg.es.select(_source=oe.target), key=lambda x: x["length"])
                for ooe in ooes:
                    if oe["length"] + ooe["length"] <= longest and v_mark[ooe.target] == "inplay":
                        v_mark[ooe.target] = "eliminated"

        for oe in oes:
            ooes = sorted(sg.es.select(_source=oe.target), key=lambda x: x["length"])
            if len(ooes) > 1:
                shortest = ooes[0].target
                if v_mark[shortest] == "inplay":
                    v_mark[shortest] == "eliminated"
            for ooe in ooes:
                if ooe["length"] < FUZZ and v_mark[ooe.target] == "inplay":
                    v_mark[ooe.target] = "eliminated"

        for oe in oes:
            if v_mark[oe.target] == "eliminated":
                e_reduce[oe.tuple] = True   # TODO: confirm revcomp edges will be also removed in the same way
            v_mark[oe.target] = "vacant"

    # Re-construct a graph
    return ig.Graph.DictList(edges=(dict(source=e["source"],
                                         target=e["target"],
                                         length=e["length"],
                                         diff=e["diff"])
                                    for e in sg.es
                                    if not e_reduce[e.tuple]),
                             vertices=None,
                             directed=True)

In [69]:
def draw_graph(sg):
    E = [e.tuple for e in sg.es]
    N = sg.vcount()
    pos = sg.layout('kk')

    edge_trace = go.Scatter(x=[i for l in [(pos[s][0], pos[t][0], None) for s, t in E] for i in l],
                            y=[i for l in [(pos[s][1], pos[t][1], None) for s, t in E] for i in l],
                            line=dict(width=0.5, color='black'),
                            mode='lines')

    shapes = [make_line(pos[s][0] + (pos[t][0] - pos[s][0]) * 0.7,
                        pos[s][1] + (pos[t][1] - pos[s][1]) * 0.7,
                        pos[t][0],
                        pos[t][1],
                        "black",
                        4,
                        "below")
              for s, t in E]
    
    reads = [sync_reads_by_id[int(sg.vs[node]["name"].split(':')[0])] for node in range(N)]
    cover_rates = [sum([unit.end - unit.start for unit in read.units]) / read.length for read in reads]   # by 359-bp units

    node_trace = go.Scatter(x=[pos[node][0] for node in range(N)],
                            y=[pos[node][1] for node in range(N)],
                            text=[f"{sg.vs[node]['name']}<br>{round(cover_rates[node] * 100, 1)}% covered" for node in range(N)],
                            mode='markers',
                            marker=dict(
                                color=[cr for cr in cover_rates],
                                showscale=False,
                                colorscale='YlGnBu',
                                reversescale=False,
                                size=10,
                                line=dict(width=2)))

    layout = go.Layout(width=800, height=700,
                       xaxis=dict(showgrid=False,
                                  zeroline=True,
                                  zerolinecolor="yellow",
                                  showticklabels=True),
                       yaxis=dict(showgrid=False,
                                  zeroline=True,
                                  zerolinecolor="yellow",
                                  showticklabels=True),
                       shapes=shapes,
                       hovermode='closest',
                       margin=go.layout.Margin(l=0, r=0, b=0, t=0),
                       showlegend=False)
    fig = go.Figure(data=[edge_trace, node_trace], layout=layout)
    py.iplot(fig)
    return pos

In [7]:
ovlps = las_to_ovlps("DMEL_CCS_CEN")

In [8]:
len(ovlps)

22972

In [21]:
cur_id_to_ori_id = {i + 1: sync_reads[i].id for i in range(len(sync_reads))}

In [23]:
ovlps = [(cur_id_to_ori_id[ovlp[0]], cur_id_to_ori_id[ovlp[1]], *ovlp[2:]) for ovlp in ovlps]

In [24]:
ovlps[:10]

[(73, 16873, 'c', 7848, 13814, 13814, 0, 5975, 12959, 37, 0.62),
 (73, 198167, 'n', 0, 9406, 13814, 2240, 11653, 11653, 15, 0.16),
 (300, 54493, 'n', 0, 4016, 13059, 8120, 12133, 12133, 6, 0.15),
 (546, 147506, 'n', 0, 7200, 12867, 5397, 12593, 12593, 18, 0.25),
 (546, 273709, 'c', 1430, 12867, 12867, 0, 11433, 13101, 57, 0.5),
 (559, 132531, 'c', 0, 8080, 14494, 4263, 12335, 12335, 12, 0.15),
 (559, 202130, 'n', 9058, 14494, 14494, 0, 5417, 14534, 30, 0.55),
 (559, 262499, 'c', 0, 4781, 14494, 8532, 13310, 13310, 5, 0.1),
 (559, 299496, 'c', 0, 8778, 14494, 4545, 13299, 13299, 40, 0.46),
 (741, 133575, 'c', 0, 4629, 12604, 6776, 11411, 11411, 22, 0.47)]

In [9]:
# Remove duplicated overlaps and filter by %diff and overlap length
ovlps = list(filter(lambda ovlp: ovlp[0] < ovlp[1] and ovlp[-1] < 1. and ovlp[4] - ovlp[3] >= 4000, ovlps))

In [25]:
len(ovlps)

1704

In [26]:
sg = transitive_reduction(construct_string_graph(ovlps))

In [27]:
cc = [(g, g.vcount()) for g in sg.clusters(mode="weak").subgraphs() if g.vcount() >= 10]

In [28]:
cc

[(<igraph.Graph at 0x2b35bf0719a8>, 10),
 (<igraph.Graph at 0x2b35bf071b88>, 11),
 (<igraph.Graph at 0x2b35bf071d68>, 10),
 (<igraph.Graph at 0x2b35bf077228>, 10),
 (<igraph.Graph at 0x2b35bf0777c8>, 16),
 (<igraph.Graph at 0x2b35bf0779a8>, 12),
 (<igraph.Graph at 0x2b35bf080318>, 51),
 (<igraph.Graph at 0x2b35bf080c78>, 12),
 (<igraph.Graph at 0x2b35bf0a0138>, 16),
 (<igraph.Graph at 0x2b35bf0a0d68>, 12),
 (<igraph.Graph at 0x2b35beeaf048>, 51),
 (<igraph.Graph at 0x2b35beeaf228>, 36),
 (<igraph.Graph at 0x2b35beeaf4f8>, 12),
 (<igraph.Graph at 0x2b35beeaf6d8>, 14),
 (<igraph.Graph at 0x2b35beeb6048>, 16),
 (<igraph.Graph at 0x2b35beeb6138>, 10),
 (<igraph.Graph at 0x2b35beeb6408>, 10),
 (<igraph.Graph at 0x2b35beeb6d68>, 36),
 (<igraph.Graph at 0x2b35beed88b8>, 11),
 (<igraph.Graph at 0x2b35beed8c78>, 14),
 (<igraph.Graph at 0x2b35beedb408>, 13),
 (<igraph.Graph at 0x2b35beedb8b8>, 11),
 (<igraph.Graph at 0x2b35beee1408>, 13),
 (<igraph.Graph at 0x2b35beee14f8>, 10),
 (<igraph.Graph 

In [70]:
draw_graph(cc[0][0])

<Layout with 10 vertices and 2 dimensions>

In [44]:
#[g[0].ecount() for g in cc]   # number of edges in each connected component

In [71]:
for g, s in cc:
    draw_graph(g)

In [45]:
# length of contig(s) in each connected component (just a rough estimate)
# NOTE: be careful also revcomp exists
s = 0
for c, n in cc:
    l = sum([e["length"] for e in c.es])
    s += l
    print(l)
print(s / 2)

39379
173222
41497
28047
241983
49646
58457
45345
48789
73690
44241
44598
34638
251775
37512
176001
44829
57496
46004
46540
44833
46635
26947
68186
44506
45952
84994
78878
52679
40517
36767
42324
51116
52646
50039
39109
43580
49240
33698
39646
1277990.5


## Conclustion

おそらくずっと pure tandem repeat ではなく、途中に cover rate の低いリードを挟まないといけない可能性が高いことが分かった。